# Proyecto 1. Calendario laLiga

In [1]:
# !pip install ortools

#### Partidos

In [2]:
import pandas as pd
from itertools import permutations

df = pd.read_csv('../Data/equipos_data.csv', index_col = 0)
equipos = list(df['equipo'].unique())
partidos = list(permutations(df['equipo'].unique(), 2))

#### Días

In [3]:
import pickle
filename = '../Data/list_calendar.pkl'
lista_dias = open(filename, 'rb')
with open(filename, 'rb') as f:
    lista_dias=pickle.load(f)
# lista_dias

### Modelo

In [4]:
filename = '../Data/predicciones.pickle'
predicciones = open(filename, 'rb')
with open(filename, 'rb') as f:
    predicciones=pickle.load(f)

## Modelado

### Variables

https://developers.google.com/optimization/assignment/assignment_example

In [5]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('GLOP')

x = {}
for i in partidos:
    for j in lista_dias:
        x[i, j] = solver.IntVar(0, 1, f'Partido {i} jugado el dia {j}')

### Variables ficticias

In [6]:
dl = {}
for i in partidos:
    dl[i] = solver.IntVar(0, len(lista_dias)-1, f'Día lectivo en el que se juega {i}')
    solver.Add(dl[i] == solver.Sum([dia * x[i,j] for dia, j in enumerate(lista_dias)]))

### Restricciones

<div class="alert alert-info">
  <strong>Máximo 4 partidos por día</strong>
</div>

In [7]:
for j in lista_dias:
    solver.Add(solver.Sum([x[i, j] for i in partidos])<=4)

<div class="alert alert-info">
  <strong>Máximo 380 partidos (hay que ponerla en el documento) </strong>
</div>

In [8]:
c_380partidos = solver.Constraint(380, 380, "380partidos_maximo")

for i in partidos:
    for j in lista_dias:
        c_380partidos.SetCoefficient(x[i,j],1)

<div class="alert alert-info">
  <strong>Todos los partidos tienen que jugarse una vez</strong>
</div>

In [9]:
for i in partidos:
    solver.Add(solver.Sum([x[i, j] for j in lista_dias])==1)

### Función objetivo

In [10]:
objective_terms = []
for i in partidos:
    for j in lista_dias:
        objective_terms.append(predicciones[i,j] * x[i, j])
solver.Maximize(solver.Sum(objective_terms))

### Resultado

In [11]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Audiencia total = {int(solver.Objective().Value())} espectadores\n')
    print(f'Audiencia media = {int(solver.Objective().Value()/380)} espectadores\n')
    cont = 0
    for i in partidos:
        for j in lista_dias:
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                cont+=1
                print(f'Partido {i} jugado el día {j}.' +
                      f' Audiencia esperada: {int(predicciones[(i,j)])} espectadores')
                
    print(f'\nPartidos totales jugados: {cont}')
    
else:
    print('Sin solución.')

Audiencia total = 13482152 espectadores

Audiencia media = 35479 espectadores

Partido ('Real Madrid CF', 'FC Barcelona') jugado el día 10-03-2023. Audiencia esperada: 82428 espectadores
Partido ('Real Madrid CF', 'Atlético de Madrid') jugado el día 27-08-2022. Audiencia esperada: 69975 espectadores
Partido ('Real Madrid CF', 'Sevilla FC') jugado el día 21-04-2023. Audiencia esperada: 75169 espectadores
Partido ('Real Madrid CF', 'Real Betis Balompié') jugado el día 14-05-2023. Audiencia esperada: 77598 espectadores
Partido ('Real Madrid CF', 'Real Sociedad') jugado el día 03-06-2023. Audiencia esperada: 75270 espectadores
Partido ('Real Madrid CF', 'Villarreal CF') jugado el día 01-04-2023. Audiencia esperada: 89682 espectadores
Partido ('Real Madrid CF', 'Athletic Club') jugado el día 26-05-2023. Audiencia esperada: 79063 espectadores
Partido ('Real Madrid CF', 'Valencia CF') jugado el día 12-02-2023. Audiencia esperada: 78380 espectadores
Partido ('Real Madrid CF', 'CA Osasuna') jug

Partido ('RCD Espanyol', 'Athletic Club') jugado el día 06-05-2023. Audiencia esperada: 35366 espectadores
Partido ('RCD Espanyol', 'Valencia CF') jugado el día 29-01-2023. Audiencia esperada: 34634 espectadores
Partido ('RCD Espanyol', 'CA Osasuna') jugado el día 20-05-2023. Audiencia esperada: 35433 espectadores
Partido ('RCD Espanyol', 'RC Celta de Vigo') jugado el día 28-04-2023. Audiencia esperada: 34492 espectadores
Partido ('RCD Espanyol', 'Rayo Vallecano') jugado el día 26-03-2023. Audiencia esperada: 29745 espectadores
Partido ('RCD Espanyol', 'Elche CF') jugado el día 13-01-2023. Audiencia esperada: 34805 espectadores
Partido ('RCD Espanyol', 'Getafe CF') jugado el día 20-01-2023. Audiencia esperada: 30038 espectadores
Partido ('RCD Espanyol', 'RCD Mallorca') jugado el día 04-03-2023. Audiencia esperada: 31714 espectadores
Partido ('RCD Espanyol', 'Cádiz CF') jugado el día 28-08-2022. Audiencia esperada: 29189 espectadores
Partido ('RCD Espanyol', 'UD Almería') jugado el día 

### Comprobación de restricciones

"results" es una matriz en el que las filas son los partidos y las columnas los día. results[i,j] = 1 si se juega el partido i el día j. Como es una matriz no se puede indexar con texto, por lo que "i" y "j" son el índice que corresponde a ese día y a es partido. Por ejemplo, el i = 0 corresponde al partido ('Real Madrid CF', 'FC Barcelona'). Si tienes el nombre del partido o el día y lo quieres pasar a índice puedes utilizar "partido_a_ind" o "dia_a_ind".

In [12]:
import pandas as pd
import numpy as np

dia_a_ind = {dia : i for i, dia in enumerate(lista_dias)}
partido_a_ind = {p : i for i, p in enumerate(partidos)}
results = np.zeros((len(partidos), len(lista_dias)))

for k,v in x.items():
    i, j = k
    i = partido_a_ind[i]
    j = dia_a_ind[j]
    
    results[i, j] = int(round(v.solution_value()))

<div class="alert alert-info">
  <strong>Día lectivo</strong>
</div>

In [13]:
bien = True
for partido, dia_lectivo in dl.items():
    i = partido_a_ind[partido]
    j = round(dia_lectivo.solution_value())

    if results[i, j] < 0.5:
        print('La restricción no va bien')
        bien = False
        break
    
if bien:
    print('La restricción funciona correctamente')

La restricción funciona correctamente


<div class="alert alert-info">
  <strong>Máximo 4 partidos por día</strong>
</div>

In [20]:
suma = np.sum(results, axis = 0) 
if np.all(suma <= 4):
    print('La restricción funciona correctamente')
else: print('La restricción no va bien')

La restricción funciona correctamente


<div class="alert alert-info">
  <strong>Máximo 380 partidos (hay que ponerla en el documento) </strong>
</div>

In [16]:
cont = 0
for i in partidos:
    for j in lista_dias:
        if x[i, j].solution_value() > 0.5:
            cont+=1
print(f'Número total de partidos jugados: {cont}')
if cont == 380:
    print('La restricción funciona correctamente')
else: print('La restricción no va bien')

Número total de partidos jugados: 380
La restricción funciona correctamente


<div class="alert alert-info">
  <strong>Todos los partidos tienen que jugarse una vez</strong>
</div>

In [19]:
suma = np.sum(results, axis = 1)
if np.all(suma > 0):
    print('La restricción funciona correctamente')
else: print('La restricción no va bien')

La restricción funciona correctamente
